In [93]:
import pandas as pd
import requests
from keys import MySQL_pass
from keys import AeroDatabox
import sqlalchemy
from IPython.display import JSON

In [95]:
def icao_airport_codes():

    schema = "gans_data" # The name of your database
    host = "127.0.0.1"
    user = "root"
    password = MySQL_pass # Your MySQL password
    port = 3306

    connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

    cities_df = pd.read_sql("cities", con=connection_string)

    list_for_df = []

    for _, city in cities_df.iterrows():
        latitude = city["Latitude"]
        longitude = city["Longitude"]
        city_id = city["city_id"]

        url = "https://aerodatabox.p.rapidapi.com/airports/search/location"

        querystring = {"lat":latitude,"lon":longitude,"radiusKm":"50","limit":"5","withFlightInfoOnly":"true"}
        headers = {
          "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com",
          "X-RapidAPI-Key": AeroDatabox #USE THE PROVIDED KEY
        }

        response = requests.get(url, headers=headers, params=querystring)

        for item in response.json()['items']:
            airports_data = {
            "city_id": city_id,
            "icao": item.get("icao", None),
            "municipality_name": item.get("municipalityName", None)
        }


        list_for_df.append(airports_data)

    airports_df = pd.DataFrame(list_for_df)

    return airports_df

In [97]:
airports_df = icao_airport_codes()
airports_df

,city_id,icao,municipality_name
0,1,EDDB,Berlin
1,2,EDDH,Hamburg
2,3,EDDM,Munich
3,4,EDDB,Berlin
4,5,EDDH,Hamburg
5,6,EDDM,Munich


### Push the "airports_to_db" to the empty "airports" table in the database

In [100]:
schema = "gans_data" # The name of your database
host = "127.0.0.1"
user = "root"
password = MySQL_pass # Your MySQL password
port = 3306

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [ ]:
airports_df.to_sql('airports',
                  if_exists='append',
                  con=connection_string,
                  index=False)

### SUCCESS!